In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lmfit import Model
import matplotlib.ticker as ticker

In [2]:
# Functions

def line(x, m, b):
    return x * m + b

def tlm(s, r_s, N, w, L, r_c):
    return ((r_s * s) / (2 * N * (w + L + s) - L - s)) + (2 * r_c)

def tlm2(s, r_s, N, w, L, l_t):
    return (r_s / (2 * N * (w + L + s) - L - s)) * (s + 2 * l_t)

def w_contact(s, N, w, L):
    return (2 * N * (w + L + s)) - L - s

In [3]:
# Models

line_mod = Model(line)
line_params = line_mod.make_params(m = 1, b = 1)

# Considering L_t to get \rho_c Contact Resistivity
tlm_mod2 = Model(tlm2)
tlm2_params = tlm_mod2.make_params()
tlm2_params['r_s'].set(value = 1e6)
tlm2_params['N'].set(value = 60, vary = False)
tlm2_params['w'].set(value = 5e-4, vary = False)
tlm2_params['L'].set(value = 3200e-4, vary = False)
tlm2_params['l_t'].set(value = 1e-5, min = 0)

In [4]:
# Settings

col = ['Potential (V)', 'Current (A)', 'Time (s)']
col2 = ['Structure', 'Spacing (cm)', 'Resistance ($\Omega$)', 'Device']
idea2 = {'P01': 2e-4,
         'P02': 5e-4,
         'P03': 10e-4,
         'P04' : 20e-4
         }

formatter = ticker.FuncFormatter(lambda x, pos: "{:.1f}".format(x / 1e6))
formatter_1 = ticker.FuncFormatter(lambda x, pos: "{:.1f}".format(x * 1e4))
formatter_2 = ticker.FuncFormatter(lambda x, pos: "{:.1f}".format(x * 1e6))
formatter_3 = ticker.FuncFormatter(lambda x, pos: "{:.1f}".format(x / 1e3))

palette = 'viridis'

In [5]:
# Function to process data

def C_R(path, thickness, sample):
    
    files = [os.path.join(path, i) for i in os.listdir(path)]
    iv_val = pd.DataFrame(columns = col2)
    iv_df = pd.DataFrame()
    n = 0
    for i in files:
        directory, filename = os.path.split(i)
        df = pd.read_csv(i, sep = ';', skiprows = 1, names = col)
        df['Device'] = filename[:-4]
        # df['dI/dV'] = np.gradient(df[col[1]], df[col[0]])
        
        for structure in idea2.keys():
            if structure in filename:
                df['Spacing ($\mu$m)'] = idea2[structure] *1e4
        
        line_fit = line_mod.fit(df[col[1]], line_params, x = df[col[0]])
        slope = line_fit.values['m']
        resistance = 1 / slope
        
        for j in idea2.keys():
            if j in filename:
                iv_val.loc[n] = (j, idea2[j], resistance, filename[:-4])
                
                
        # df['Norm dI/dV'] = df['dI/dV'] / slope
        iv_df = pd.concat([iv_df, df], ignore_index = True)
        
        n += 1
    iv_df['Sample'] = sample
    iv_val['Sample'] = sample
    # print(iv_df)

    width_ = (w_contact(iv_val[col2[1]], 60, 5e-4, 3200e-4))
    width_avg = width_.mean()

    all_avg = pd.DataFrame()
    u_s = iv_val.Structure.unique()
    df_dict = {i : pd.DataFrame() for i in u_s}
    
    for key in df_dict.keys():
        df_dict[key] = iv_val[:][iv_val.Structure == key]
        
    for key, df in df_dict.items():
        mean_ = df.mean(numeric_only = True).reset_index().transpose()
        mean_.columns = mean_.iloc[0]
        mean_ = mean_[1:].reset_index(drop=True)
        all_avg = pd.concat([all_avg, mean_], ignore_index = True)
    
    
    tlm_fit2 = tlm_mod2.fit(all_avg[col2[2]], tlm2_params, s = all_avg[col2[1]])   
    # using the average for the fit. Same values, lower numerical error 
    # tlm_fit2 = tlm_mod2.fit(iv_val[col2[2]], tlm2_params, s = iv_val[col2[1]])
    
    # print(tlm_fit2.fit_report())
    rc = tlm_mod2.eval(tlm_fit2.params, s = 0) / 2
    lt = tlm_fit2.values['l_t']
    p_c = rc * lt * width_avg #Contact resistiviy ohm * cm^2
    # sigma = (1 / tlm_fit2.values['r_s'] ) / (thickness * 1e-7 * width_avg)
    sigma = 1 / (tlm_fit2.values['r_s'] * thickness * 1e-7)
    r_s = tlm_fit2.values['r_s']

    print('Contact Resistivity = ' + str(p_c) + ' Ohm cm2')
    print('LMO conductivity = ' + str(sigma) + ' S/cm')
    
    
    # Plot
    fig, ax = plt.subplots(1, 2, figsize = (15,5), dpi = 200) #    1, 2, figsize = (15,5),
    
    cross_zero = np.array([-2, 0, 20]) * 1e-4
    y_zero = tlm_mod2.eval(tlm_fit2.params, s = cross_zero)
    
    sns.lineplot(data = iv_val, x = col2[1], y = col2[2], marker='o', markersize = 10,errorbar=('sd', 2) ,err_style='bars', linestyle='', err_kws={'ecolor': 'black', 'capsize': 2})
    sns.lineplot(x = cross_zero, y = y_zero)
    #lineplot with error bars showing 2 STD
    
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)  # Horizontal zero line
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)  # Vertical zero line
    
    ax[1].set_xlabel('Electrode Spacing (µm)')
    ax[1].set_ylabel('Resistance ($\Omega$)')
    # ax[1].yaxis.set_major_formatter(formatter)
    ax[1].xaxis.set_major_formatter(formatter_1)
    #
    sns.scatterplot(data = iv_val, x = col2[1], y = col2[2], ax = ax[0])
    
    fig, ax = plt.subplots(dpi = 200)
    sns.lineplot(data = iv_df, x = 'Potential (V)', y = 'Current (A)', hue = 'Spacing ($\mu$m)')
    ax.yaxis.set_major_formatter(formatter_2)
    ax.set_ylabel('Current ($\mu$A)')
    
    fit_vals = pd.DataFrame({'Sheet Resistance' : [r_s],
                'Contact Resistance' : [rc],
                'Transfer Lenght' : [lt],
                'Contact Resistivity' : [p_c],
                'Conductivity' : [sigma],
                'Sample' : [sample]})
    
    result = iv_val, tlm_fit2, fit_vals, iv_df
    
    return result

In [ ]:
# TI2_03 LMO 8h Pt
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_Pt'
sample = 'LMO 350nm/Pt 100nm'
lmo8h_pt = C_R(path, 350, sample) # , lmo8h_pt_fit, lmo8h_pt_fit_vals, lmo8h_pt_iv_curves
print(lmo8h_pt[1].fit_report())

In [ ]:
#LMO8h Pt Sample 2
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_Pt_20231029'
sample = 'LMO 350nm/Pt 100nm 2'
lmo8h_pt2 = C_R(path, 350, sample)
print(lmo8h_pt2[1].fit_report())

In [ ]:
# LMO8h Pt 1 and 2
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_Pt_1_2'
sample = 'Pt'
lmo8h_pt12 = C_R(path, 350, sample)
print(lmo8h_pt12[1].fit_report())


In [ ]:
#LMO8h ITO
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_ITO_20231029'
sample = 'ITO'
lmo8h_ito = C_R(path, 350, sample)
print(lmo8h_ito[1].fit_report())

In [ ]:
# LMO8h Au
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_Au_20231106\1'
sample = 'LMO 350/Au'
lmo8h_au = C_R(path, 350, sample)
print(lmo8h_au[1].fit_report())


In [ ]:
# LMO8h Au2
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_Au_20231106\2'
sample = 'Au'
lmo8h_au2 = C_R(path, 350, sample)
print(lmo8h_au2[1].fit_report())


In [ ]:
# # LMO8h Au 1 2
# path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_Au_20231106\both'
# sample = 'LMO 350/Au'
# lmo8h_au12 = C_R(path, 350, sample)
# print(lmo8h_au12[1].fit_report())

In [ ]:
# LMO8h Al
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_Al_220231106'
sample = 'Al'
lmo8h_al = C_R(path, 350, sample)

df5um = lmo8h_al[3].loc[lmo8h_al[3]['Spacing ($\\mu$m)'] == 5.0]


In [ ]:
fig, ax = plt.subplots(dpi = 100)
sns.lineplot(data = df5um, x = 'Potential (V)', y = 'Current (A)')
ax.yaxis.set_major_formatter(formatter_2)
ax.set_ylabel('Current (µA)')

In [ ]:
# LMO8h Cr
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_Cr'
sample = 'Cr'
lmo8h_cr = C_R(path, 350, sample)
print(lmo8h_cr[1].fit_report())

In [ ]:
# LMO8h Mo
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Contact Resistance\IDEA2_TLM\LMO8h_Mo'
sample = 'Mo'
lmo8h_mo = C_R(path, 350, sample)
print(lmo8h_mo[1].fit_report())

In [ ]:
# Compare
# sns.set_palette('Set2')
fig, ax = plt.subplots(dpi = 200)
samples = [lmo8h_pt12, lmo8h_ito, lmo8h_au2, lmo8h_cr, lmo8h_mo]
vals = pd.DataFrame()
fit_v = pd.DataFrame()
cross_zero = np.array([-2, 0, 20]) * 1e-4
for i in samples:
    vals = pd.concat([vals, i[0]], ignore_index = True)
    fit_v = pd.concat([fit_v, i[2]], ignore_index = True)
    y_zero = tlm_mod2.eval(i[1].params, s = cross_zero)
    sns.lineplot(x = cross_zero, y = y_zero)
sns.lineplot(data = vals, x = col2[1], y = col2[2], hue = 'Sample', marker='o', markersize = 8, errorbar='sd',err_style='bars', linestyle='', legend = False) #, err_kws={'ecolor': 'red', 'capsize': 2}
plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)  # Horizontal zero line
plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)  # Vertical zero 
# ax.legend(False)
ax.xaxis.set_major_formatter(formatter_1)
ax.yaxis.set_major_formatter(formatter_3)
ax.set_xlabel('Spacing (µm)')
ax.set_ylabel('Resistance (k$\Omega$)')


# fig, ax = plt.subplots(2, 2, figsize = (15,10), dpi = 100)
fig, ax = plt.subplots(dpi = 200)
# sns.barplot(data = fit_v, x = 'Sample', y = 'Conductivity', ax = ax[0,0])
sns.barplot(data = fit_v, x = 'Sample', y = 'Contact Resistivity')
plt.ylabel('Specific Contact Resistance ($\Omega$ cm$^2$)')
plt.xlabel('Metal')
# ax[0,0].set_xticklabels(ax[0,0].get_xticklabels(), rotation=15)
# ax[0,1].set_xticklabels(ax[0,0].get_xticklabels(), rotation=15)
# sns.barplot(data = fit_v)
